In [ ]:
import re
import torch
import json

from transformers import BertForTokenClassification, BertTokenizer, BertTokenizerFast, BatchEncoding
from tokenizers import decoders, Encoding
from alignment import align_tokens_and_annotations_bilou

from prepareData import prepare_data

In [ ]:
model = BertForTokenClassification.from_pretrained("bert_ner_finetuned_iliad-with-gpu-pattern2.model")

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

In [ ]:
label_list = ['O', 'B-CLEntity', 'I-CLEntity', 'L-CLEntity', 'U-CLEntity']

In [ ]:
odyssey_lines = []
odyssey = open("../example-texts/odyssey.txt")
for line in odyssey:
    line = line.strip()
    odyssey_lines.append(line)

In [ ]:
odyssey_lines = [line for line in odyssey_lines if line]

In [ ]:
odyssey_lines[:10]

In [ ]:
pred = []
for line in odyssey_lines:
    tokens_line = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(line)))
    inputs_line = tokenizer.encode(line, return_tensors="pt")
    outputs_line = model(inputs_line).logits
    predictions_line = torch.argmax(outputs_line, dim=2)
    pred_line_label = []
    for prediction in predictions_line[0].numpy():
        pred_line_label.append(label_list[prediction])
    pred.append(pred_line_label)
    
with open('odyssey_ner_predictions.txt', 'w') as f:
    f.write(json.dumps(pred))

In [ ]:
# with open('odyssey_ner_predictions.txt', 'r') as f:
#     a = json.loads(f.read())
# print(a)

In [ ]:
# # Evaluation
# pattern = r'(\b[A-Z][a-z]+\b)(\s\b[A-Z][a-z]+\b)*'
# odyssey_regex_find = prepare_data("../example-texts/odyssey.txt", pattern)

In [ ]:
# # For testing the result. Need a way to test whether tuned bert is accurately identifying CLEntities
# odyssey_regex_find[:10] 